In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import english_words

[nltk_data] Downloading package stopwords to C:\Users\yousef
[nltk_data]     salem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [4]:
train_df['Title'] = train_df['Title'].apply(preprocess_text)
train_df['Description'] = train_df['Description'].apply(preprocess_text)


test_df['Title'] = test_df['Title'].apply(preprocess_text)
test_df['Description'] = test_df['Description'].apply(preprocess_text)

In [5]:
cv = CountVectorizer()
X_train = cv.fit_transform(train_df['Title'] + ' ' + train_df['Description'])
y_train = train_df['Class Index']
X_test = cv.transform(test_df['Title'] + ' ' + test_df['Description'])
y_test = test_df['Class Index']

In [6]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

MultinomialNB()

In [7]:
y_pred = nb_classifier.predict(X_test)

print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           1       0.91      0.90      0.90      1900
           2       0.95      0.98      0.97      1900
           3       0.87      0.85      0.86      1900
           4       0.87      0.88      0.88      1900

    accuracy                           0.90      7600
   macro avg       0.90      0.90      0.90      7600
weighted avg       0.90      0.90      0.90      7600



In [12]:
ps = PorterStemmer()
flag = False
user_title = input("enter the title of the article: ")
user_description = input('enter the description of the article: ')
if all([token in english_words.get_english_words_set(['web2']) for token in list(filter( None,ps.stem(user_title).split()))]):
    user_title = user_title
    flag = True
else:
    user_title = ''
    print('invalid title')
    flag = False
if all([token in english_words.get_english_words_set(['web2']) for token in list(filter(None, ps.stem(user_description).split()))]):
    user_description = user_description
    flag = True
else:
    user_description = ''
    print('invalid description') 
    flag = False
if flag == True:
    user_article = {'title': str(user_title), 'description': str(user_description)}
    user_article = pd.DataFrame(user_article, index=[0])
    user_article['title'] = user_article['title'].apply(preprocess_text)
    user_article['description'] = user_article['description'].apply(preprocess_text)
    user_input = cv.transform(user_article['title'] + ' ' + user_article['description'])
    y_pred = nb_classifier.predict(user_input.toarray())
    if y_pred == 1:
        print('world')
    elif y_pred == 2:
        print('sports')
    elif y_pred == 3:
        print('business')
    elif y_pred == 4:
        print('tech')
    else:
        print('unknown')

enter the title of the article: hello world 
enter the description of the article: hello world 
tech
